In [1]:

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))
        
from sklearn.compose import ColumnTransformer        
from sklearn.preprocessing import LabelEncoder,OneHotEncoder,MinMaxScaler
from lightgbm import LGBMClassifier
from sklearn.linear_model import LogisticRegression
from lightgbm import LGBMClassifier

/kaggle/input/titanic/train.csv
/kaggle/input/titanic/test.csv
/kaggle/input/titanic/gender_submission.csv


In [2]:
train_dataset=pd.read_csv('../input/titanic/train.csv')
test_dataset=pd.read_csv('../input/titanic/test.csv')

In [3]:
test_dataset.head()


,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S


In [4]:
train_dataset=train_dataset.drop("Cabin",axis=1)
test_dataset=test_dataset.drop("Cabin",axis=1)
train_dataset=train_dataset.drop("Ticket",axis=1)
test_dataset=test_dataset.drop("Ticket",axis=1)
train_dataset=train_dataset.drop("Name",axis=1)
test_dataset=test_dataset.drop("Name",axis=1)
train_dataset=train_dataset.drop("Fare",axis=1)
test_dataset=test_dataset.drop("Fare",axis=1)
train_dataset=train_dataset.drop("Embarked",axis=1)
test_dataset=test_dataset.drop("Embarked",axis=1)
train_dataset.head()

,PassengerId,Survived,Pclass,Sex,Age,SibSp,Parch
0,1,0,3,male,22.0,1,0
1,2,1,1,female,38.0,1,0
2,3,1,3,female,26.0,0,0
3,4,1,1,female,35.0,1,0
4,5,0,3,male,35.0,0,0


In [5]:
train_dataset.dropna(inplace=True)
test_dataset.dropna(inplace=True)


In [6]:
y=train_dataset['Survived']
col='Survived'
x = train_dataset.loc[:, train_dataset.columns != col]


In [7]:
#ct1=ColumnTransformer(transformers=[('encoder',OneHotEncoder(),[2])],remainder='passthrough')
#x1=np.array(ct1.fit_transform(x))
#ct2=ColumnTransformer(transformers=[('encoder',OneHotEncoder(),[1])],remainder='passthrough')
#x2=np.array(ct2.fit_transform(x1))
#ct3=ColumnTransformer(transformers=[('encoder',OneHotEncoder(),[6])],remainder='passthrough')
#x2=np.array(ct3.fit_transform(x2))

ct = ColumnTransformer(transformers=[('encoder', OneHotEncoder(), [2])], remainder='passthrough')
ct.fit(x)
x1=np.array(ct.transform(x))
ct2 = ColumnTransformer(transformers=[('encoder', OneHotEncoder(), [2])], remainder='passthrough')
ct2.fit(test_dataset)
x2=np.array(ct2.transform(test_dataset))


In [8]:
#scaler=MinMaxScaler()
#x_train_norm=scaler.fit_transform(x1)
#x_test_norm=scaler.fit_transform(x2)

In [9]:
classifierLGBM=LGBMClassifier(n_estimators=300,random_state=0)
classifierLGBM.fit(x1,y)


LGBMClassifier(n_estimators=300, random_state=0)

In [10]:

y_predLGBM=classifierLGBM.predict(x2)

In [11]:
y_predLGBM = pd.DataFrame(y_predLGBM,columns=['Survived'])
sub = pd.concat([test_dataset.PassengerId.astype(str),y_predLGBM],axis=1)
sub.set_index('PassengerId',inplace=True)
sub.to_csv(f"submission.csv")
sub.shape
#y_predLGBM = pd.DataFrame(y_predLGBM,columns=['Survived'])
#sub = pd.concat([test_dataset.PassengerId,y_predLGBM],axis=1)
#sub.set_index('PassengerId',inplace=True)
#sub.to_csv(f"submission.csv")

(402, 1)

In [12]:
sub.head()

,Survived
PassengerId,
892,0.0
893,0.0
894,0.0
895,0.0
896,0.0
